In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP as abatement
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

*Specify settings:*

In [2]:
name = 'A1_calib_test'
f = {'tech': directory['data']+"/techdata_dors_2.xlsx",
     'other': directory['data']+"/Othdata_dors_2.xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

## 1: Initialize model

*Initialize with ID technologies:*

In [5]:
gm = abatement.abate(nts=nts,tech=tech,work_folder=directory['work'],use_EOP=True,**{'data_folder':directory['gams'],'name': name})

In [6]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)

*New mappings:*
* M2C[z,n]: Mapping from emission type to component in EOP. 
* EOP\_i2ai[n,nn]: Mapping to aggregate use of inputs in EOP. 

*Adding the EOP module comes with two types of adjustments: First, there are completely new block of equations to be written. For instance, the solution to the tree structure problem. Second, there are adjustments that replace equations that are applied in the simple ID case:*

To do:
* Keep 'ID\_sum', 'ID\_Em', 'ID\_calib\_aux'.
* Add EOP module that computes PwThat for EOP inputs (similar to ID). 
* Add EOP-ID module that computes aggregate use of inputs (qD[ai]) and the effective price of emissions (pMhat). In the model blocks, replace the ID\_agg with this new one that does both.
* Add EOP module that computes currapp.
* Add specific EOP module that computes post-abatement emissions, the effective price on emissions and the amount of abated emissions.
* Add EOP-ID module that computes the minobj for calibration purposes